In [2]:
# imports
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
#From city bikes workbook
path = '/Users/parkerharalds/Documents/Lighthouse/Statistical_Modelling_Project/data/city_bikes.csv'
df = pd.read_csv(path)

In [ ]:
df2 = df.head(2)
df2

**Function to make API calls based on latitude and longitude from city_bikes.csv:**

In [94]:
api_key = os.environ['FS_API_KEY']
headers = {"accept": "application/json",
           'Authorization': api_key}
url ='https://api.foursquare.com/v3/places/search?'

def get_nearby_places(lat, lon):
    filters = {  'll': f'{lat},{lon}',            #lat and lon to same string
                'radius': 1000,                  # Radius in meters (specified above)  
                'categories': 13032,       # code for coffee shops and cafes
                'limit': 50              
            }                 
    response = requests.get(url, headers= headers, params= filters)
    if response.status_code == 200:             #if api call was successfull return the results
        data = response.json()
        results = data['results']
        df_results = pd.json_normalize(results)
        return df_results
    else:
        print('Error')
        return None

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [95]:
# Iterate through the DataFrame and make API calls
df_all_stations = pd.DataFrame()
for index, row in df2.iterrows():
    lat, lon = row['latitude'], row['longitude']
    df_this_station = get_nearby_places(lat, lon)
    
    if df_this_station is not None:
        df_this_station['station'] = row['station_id']
        df_all_stations = pd.concat([df_all_stations, df_this_station], axis=0)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [101]:
yelp_api_key = os.environ['yelp_api']
headers = {"accept": "application/json",
           'Authorization': yelp_api_key}
filters = { 'latitude': 48.42, 
                'longitude': 123.36,           
                'radius': 1000,                    
                'categories': 'cafes'   
            }      
url = 'https://api.yelp.com/v3/businesses/search'
response = requests.get(url, headers= headers, params= filters)
response

<Response [400]>

In [98]:
yelp_api_key = os.environ['yelp_api']
headers = {"accept": "application/json",
           'Authorization': yelp_api_key}
url = 'https://api.yelp.com/v3/businesses/search?'

def get_yelp_places(lat, lon):
    filters = { 'latitude': lat, 
                'longitude': lon,           
                'radius': 1000,                    
                'categories': 'cafes',      
            }                 
    response = requests.get(url, headers= headers, params= filters)
    if response.status_code == 200:             #if api call was successfull return the results
        data = response.json()
        results = data['results']
        df_results = pd.json_normalize(results)
        return df_results
    else:
        print('Error')
        return None


In [99]:

# Iterate through the DataFrame and make API calls
df_yelp_stations = pd.DataFrame()
for index, row in df2.iterrows():
    latitude = row['latitude']
    longitude =  row['longitude']
    df_this_yelp = get_yelp_places(lat, lon)
    
    if df_this_yelp is not None:
        df_this_yelp['station'] = row['station_id']
        df_yelp_stations = pd.concat([df_yelp_stations, df_this_yelp], axis=0)

Error
Error


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating